In [54]:
# Importing the required libraries
import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import norm,skew

### Importing the monthly details of the Mandi and the prices 

In [55]:
df_monthly = pd.read_csv('Monthly_data_cmo.csv',parse_dates=['date'])

In [56]:
df_monthly.head()

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name,state_name
0,Ahmednagar,Bajri,2015,April,79,1406,1538,1463,2015-04-01,Ahmadnagar,Maharashtra
1,Ahmednagar,Bajri,2016,April,106,1788,1925,1875,2016-04-01,Ahmadnagar,Maharashtra
2,Ahmednagar,Wheat(Husked),2015,April,1253,1572,1890,1731,2015-04-01,Ahmadnagar,Maharashtra
3,Ahmednagar,Wheat(Husked),2016,April,387,1750,2220,1999,2016-04-01,Ahmadnagar,Maharashtra
4,Ahmednagar,Sorgum(Jawar),2015,April,3825,1600,2200,1900,2015-04-01,Ahmadnagar,Maharashtra


In [57]:
df_monthly.shape

(62429, 11)

### The dataset has three numerical features against the commodity and the type of crop.
### Let's see the stats of the dataset to look at mean, std, quartiles and identify possible outliers

In [58]:
df_monthly.describe()

,Year,arrivals_in_qtl,min_price,max_price,modal_price
count,62429.000000,6.242900e+04,6.242900e+04,6.242900e+04,62429.000000
mean,2015.337503,6.043088e+03,2.945228e+03,3.688814e+03,3296.003989
std,0.690451,3.470331e+04,1.318396e+04,7.662962e+03,3607.792534
min,2014.000000,1.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,2015.000000,3.800000e+01,1.250000e+03,1.600000e+03,1450.000000
50%,2015.000000,2.110000e+02,1.976000e+03,2.797000e+03,2425.000000
75%,2016.000000,1.364000e+03,3.900000e+03,4.647000e+03,4257.000000
max,2016.000000,1.450254e+06,3.153038e+06,1.600090e+06,142344.000000


### The above dataset shows extremely high maximum values as compared to the 75 percentiles values for all the numerical features except Year. However, the dataset has to be treated differently for different commodities. We will have to analyse the mean, std, inter quartile range and major, mild outliers for each commodity individually.

### Let's look at the percentage of missing values held by the data


In [59]:
missing_yearly = (df_monthly.isnull().sum()/len(df_monthly))*100
missing_yearly.sort_values(ascending=False)

state_name         0.0
district_name      0.0
date               0.0
modal_price        0.0
max_price          0.0
min_price          0.0
arrivals_in_qtl    0.0
Month              0.0
Year               0.0
Commodity          0.0
APMC               0.0
dtype: float64

###  The data set appears to have no missing values
### Let's bring out the numerical features and categorical data separately

In [60]:
df_monthly.drop('state_name',axis=1,inplace=True) ### dropping state name as it is Maharashtra for all rows 
numeric_features = (df_monthly.dtypes[df_monthly.dtypes!='object']).index
categric_features = (df_monthly.dtypes[df_monthly.dtypes=='object']).index

### coverting all the strings into lower case so as to perform correct grouping and aggregation in later stages

In [61]:
for i in range(0,len(categric_features)):df_monthly[categric_features[i]]=df_monthly[categric_features[i]].str.lower()
df_monthly.head()

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name
0,ahmednagar,bajri,2015,april,79,1406,1538,1463,2015-04-01,ahmadnagar
1,ahmednagar,bajri,2016,april,106,1788,1925,1875,2016-04-01,ahmadnagar
2,ahmednagar,wheat(husked),2015,april,1253,1572,1890,1731,2015-04-01,ahmadnagar
3,ahmednagar,wheat(husked),2016,april,387,1750,2220,1999,2016-04-01,ahmadnagar
4,ahmednagar,sorgum(jawar),2015,april,3825,1600,2200,1900,2015-04-01,ahmadnagar


In [62]:
#data = numeric_data.drop(['Year','date'],axis=1)
#for i in range(0,len(data.columns)):
#    plt.figure()
#    s = data.columns[i]
#    sns.distplot(data[s])
#    mu,sigma = norm.fit(data[s])
#    plt.xlabel(s)
#    plt.title('$\mu$: '+str(mu)+', $\sigma$: '+str(sigma))

In [63]:
"""def outlier_in(s):
    return ~s.between(outer_fence_l[s],outer_fence_h[s])
def replace_outlier_in(group):
    mean,std = group.mean(),group.std()
    outliers = (group-mean).abs()>3.0*std
    group[outliers]=mean
    return group
"""

'def outlier_in(s):\n    return ~s.between(outer_fence_l[s],outer_fence_h[s])\ndef replace_outlier_in(group):\n    mean,std = group.mean(),group.std()\n    outliers = (group-mean).abs()>3.0*std\n    group[outliers]=mean\n    return group\n'

### Let's now look at the groupwise statistics and try to bring out the characteristics

In [64]:
def upper_fence(x) :
    qr1 = np.percentile(x,25)
    qr3 = np.percentile(x,75)
    iqr = qr3 - qr1
    return(qr3 + 3.0*iqr)

def lower_fence(x) : 
    qr1 = np.percentile(x,25)
    qr3 = np.percentile(x,75)
    iqr = qr3 - qr1
    return(qr1 - 3.0*iqr)


In [65]:
group_stats = pd.pivot_table(df_monthly.drop(['Year'],axis=1),index='Commodity',aggfunc=['mean','std',upper_fence,lower_fence])
group_stats.head()

C:\Users\vipul\Anaconda3\lib\site-packages\pandas\core\reshape\pivot.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return concat(pieces, keys=keys, axis=1)


mean                                         \
            arrivals_in_qtl    max_price    min_price  modal_price   
amba koy         228.333333  3875.000000  2791.666667  3333.333333   
ambat chuka       33.617647  1230.470588   356.000000   451.117647   
amla              47.916667  2117.791667  1079.000000  1616.875000   
apple           9288.000000  9400.836910  5296.171674  7459.866953   
arvi            1517.148148  2420.129630  1947.203704  2181.277778   

                        std                                         \
            arrivals_in_qtl    max_price    min_price  modal_price   
amba koy         182.368674  1718.465886  1612.128510  1664.582030   
ambat chuka       40.647796   426.237467   412.511010   500.528615   
amla              65.094524   582.228252   314.838261   323.711400   
apple          27922.066896  4235.892010  3237.663334  3969.846834   
arvi            2012.683494   718.575925   641.646302   642.443167   

                upper_fence                                     lower_fence  \
            arrivals_in_qtl max_price min_price modal_price arrivals_in_qtl   
amba koy             815.00   9875.00   8375.00     9125.00          -392.5   
ambat chuka          121.00   3120.00   3133.25     3324.25           -71.5   
amla                 219.75   5034.75   2213.25     3012.75          -160.0   
apple              14693.00  26309.00  15188.00    20727.00        -10633.0   
arvi                8181.00   6227.25   5896.25     5873.75         -5920.5   

                                             
            max_price min_price modal_price  
amba koy     -1937.50  -2562.50    -2250.00  
ambat chuka   -660.00  -2284.75    -2377.25  
amla          -782.25   -151.00      183.00  
apple        -8012.00  -5441.00    -6636.00  
arvi         -1360.75  -1928.00    -1364.25

### If there is any commodity for which $\sigma=\infty$, then we can drop these. This is so because we are trying to perform time series analysis and $\sigma=\infty$ means a single data point. A single data point cannot be used for time series analysis

In [66]:
commodities_zero_std = group_stats['std']['modal_price'].index[group_stats['std']['modal_price'].apply(np.isnan)]

In [67]:
len(commodities_zero_std)

11

### there are 11 commodities which have $\sigma = \infty$ which are named below

In [68]:
commodities_zero_std

Index(['baru seed', 'ghee', 'groundnut pods (wet)', 'jui', 'maize (corn.)',
       'other spices', 'papnas', 'shahale', 'skin & bones', 'terda',
       'tuljapuri'],
      dtype='object')

###  Dropping the above mentioned points

In [69]:
indexes = []
for i in range(0,len(commodities_zero_std)):
    indexes.append((df_monthly[df_monthly['Commodity']==commodities_zero_std[i]].index)[0])
df_monthly.loc[indexes]

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name
32067,nanded,baru seed,2016,june,2,4800,4800,4800,2016-06-01,nanded
39158,jintur,ghee,2015,june,20,6100,6925,6500,2015-06-01,parbhani
62000,pune-manjri,groundnut pods (wet),2016,november,9,4167,4500,4267,2016-11-01,pune
45690,pune,jui,2014,september,3,46667,60000,53333,2014-09-01,pune
387,ahmednagar,maize (corn.),2016,august,307,762,1192,977,2016-08-01,ahmadnagar
9872,sillod,other spices,2016,october,1,5800,6000,5900,2016-10-01,aurangabad
5905,amarawati,papnas,2015,december,3,15,20,17,2015-12-01,amaravathi
61582,nashik,shahale,2016,november,2258,1745,2418,2129,2016-11-01,nasik
53401,akluj,skin & bones,2016,may,95,300,700,700,2016-05-01,solapur
45692,pune,terda,2014,september,4,3667,6000,4833,2014-09-01,pune


In [70]:
"""def replace(group):
    mean,std = group.mean(),group.std()
    outliers = (group-mean).abs()>3*std
    group[outliers]=mean
    return group"""

'def replace(group):\n    mean,std = group.mean(),group.std()\n    outliers = (group-mean).abs()>3*std\n    group[outliers]=mean\n    return group'

In [71]:
df_monthly.drop(indexes,inplace=True)

In [72]:
df_monthly.shape

(62418, 10)

### Regenerating the statistics for each commodity after dropping the commodity with single data points

In [73]:
group_stats = pd.pivot_table(df_monthly.drop(['Year'],axis=1),index='Commodity',aggfunc=['mean','std',upper_fence,lower_fence])
group_stats.head()

mean                                         \
            arrivals_in_qtl    max_price    min_price  modal_price   
Commodity                                                            
amba koy         228.333333  3875.000000  2791.666667  3333.333333   
ambat chuka       33.617647  1230.470588   356.000000   451.117647   
amla              47.916667  2117.791667  1079.000000  1616.875000   
apple           9288.000000  9400.836910  5296.171674  7459.866953   
arvi            1517.148148  2420.129630  1947.203704  2181.277778   

                        std                                         \
            arrivals_in_qtl    max_price    min_price  modal_price   
Commodity                                                            
amba koy         182.368674  1718.465886  1612.128510  1664.582030   
ambat chuka       40.647796   426.237467   412.511010   500.528615   
amla              65.094524   582.228252   314.838261   323.711400   
apple          27922.066896  4235.892010  3237.663334  3969.846834   
arvi            2012.683494   718.575925   641.646302   642.443167   

                upper_fence                                     lower_fence  \
            arrivals_in_qtl max_price min_price modal_price arrivals_in_qtl   
Commodity                                                                     
amba koy             815.00   9875.00   8375.00     9125.00          -392.5   
ambat chuka          121.00   3120.00   3133.25     3324.25           -71.5   
amla                 219.75   5034.75   2213.25     3012.75          -160.0   
apple              14693.00  26309.00  15188.00    20727.00        -10633.0   
arvi                8181.00   6227.25   5896.25     5873.75         -5920.5   

                                             
            max_price min_price modal_price  
Commodity                                    
amba koy     -1937.50  -2562.50    -2250.00  
ambat chuka   -660.00  -2284.75    -2377.25  
amla          -782.25   -151.00      183.00  
apple        -8012.00  -5441.00    -6636.00  
arvi         -1360.75  -1928.00    -1364.25

In [74]:
commodity_index = np.array(group_stats.index)

In [75]:
commodity_index[]

SyntaxError: invalid syntax (<ipython-input-75-c0675eba4d08>, line 1)